In [2]:
# =============================================================================
# =============================================================================
# # 문제 09 유형(DataSet_09.csv 이용)
#
# 구분자 : comma(“,”), 2000 Rows, 16 Columns, UTF-8 인코딩
#
# 항공사에서 고객만족도 조사를 하고 서비스 개선에 활용하고자
# 아래와 같은 데이터를 준비하였다.
#
# 컬 럼 / 정 의 / Type
# satisfaction / 서비스 만족 여부 / String
# Gender / 성별 / String
# Age / 나이 / Double
# Customer_Type / 고객 타입 / String
# Class / 탑승 좌석 등급 / String
# Flight_Distance / 비행 거리 / Double
# Seat_comfort / 좌석 안락도 점수 / Double
# Food_and_Drink / 식사와 음료 점수 / Double
# Inflight_wifi_service / 기내 와이파이 서비스 점수 / Double
# Inflight_entertainment / 기내 엔터테인먼트 서비스 점수 / Double
# Onboard_service / 탑승 서비스 점수 / Double
# Leg_room_service / 다리 공간 점수 / Double
# Baggage_handling / 수하물 취급 점수 / Double
# Cleanliness / 청결도 점수 / Double
# Departure_Daly_in_Minutes / 출발 지연 (분) / Double
# Arrival_Delay_in_Minutes / 도착 지연 (분) / Double
# =============================================================================
# =============================================================================

# =============================================================================
# (참고)
# #1
# import pandas as pd
# import numpy as np
# #2
# import scipy.stats as stats
# #3
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn import metrics
# 
# =============================================================================

In [1]:
import pandas as pd
import numpy as np

In [4]:
dataset = pd.read_csv('Dataset/Dataset_09.csv')

In [3]:
# =============================================================================
# 1.데이터 타입을 위 표에 정의된 타입으로 전처리를 한 후, 데이터 파일 내에 결측값은
# 총 몇 개인가? (답안 예시) 1
# =============================================================================

In [5]:
q1 =dataset.copy()

In [7]:
q1.dtypes

satisfaction                   object
Gender                         object
Age                           float64
Customer_Type                  object
Class                          object
Flight_Distance               float64
Seat_comfort                  float64
Food_and_drink                float64
Inflight_wifi_service         float64
Inflight_entertainment        float64
Onboard_service               float64
Leg_room_service              float64
Baggage_handling              float64
Cleanliness                   float64
Departure_Delay_in_Minutes    float64
Arrival_Delay_in_Minutes      float64
dtype: object

In [11]:
q1.isnull().sum().sum()

5

In [25]:
# 독립성 검정 카이스퀘어 검정 => crosstab 먼저 진행 -> crosstab을 chi인자로 넣는다
# =============================================================================
# 2.다음에 제시된 데이터 처리를 하고 카이제곱 독립성 검정을 수행하시오.
# - 결측값이 있다면 해당 행을 제거하시오.
# - 나이는 20 이하이면 10, 30 이하이면 20, 40 이하이면 30, 50 이하이면 40, 60 이하이면 50, 
# 60 초과는 60으로 변환하여 Age_gr으로 파생변수를 생성하시오.
# - Age_gr, Gender, Customer_Type, Class 변수가 satisfaction에 영향이 있는지 카이제곱
# 독립성 검정을 수행하시오. 
# - 연관성이 있는 것으로 파악된 변수의 검정통계량 추정치를 정수 부분만 기술하시오. 
# (답안 예시) 123
# =============================================================================

In [17]:
q2 = dataset.copy().dropna()
q2.head(3)

,satisfaction,Gender,Age,Customer_Type,Class,Flight_Distance,Seat_comfort,Food_and_drink,Inflight_wifi_service,Inflight_entertainment,Onboard_service,Leg_room_service,Baggage_handling,Cleanliness,Departure_Delay_in_Minutes,Arrival_Delay_in_Minutes
0,satisfied,Female,19.0,Loyal Customer,Eco,1981.0,4.0,4.0,1.0,4.0,4.0,4.0,4.0,4.0,3.0,28.0
1,dissatisfied,Female,64.0,Loyal Customer,Eco,2847.0,1.0,1.0,1.0,2.0,5.0,3.0,4.0,4.0,111.0,133.0
2,dissatisfied,Male,55.0,Loyal Customer,Eco,3790.0,1.0,1.0,2.0,1.0,4.0,4.0,4.0,4.0,2.0,15.0


In [18]:
q2['Age_gr'] = np.where(q2.Age <= 20,10,
                       np.where(q2.Age <= 30,20,
                           np.where(q2.Age <= 40,30,
                                    np.where(q2.Age <= 50, 40,
                                             np.where(q2.Age <= 60,50,60)
                       ))))

In [20]:
import scipy.stats as sc

In [37]:
x_var = ['Age_gr', 'Gender', 'Customer_Type', 'Class']
tmp = []
for i in x_var:
    cross_tab = pd.crosstab(index=q2[i], columns=q2.satisfaction)
    pvalue = sc.chi2_contingency(cross_tab)[1]
    static = sc.chi2_contingency(cross_tab)[0]
    tmp += [[i,pvalue,static]]
    
df = pd.DataFrame(tmp)
df.columns = ['name','pvalue','static']


In [40]:
int(df[df.pvalue < 0.05]['static'])

1066

In [5]:
# logistic 회귀
# =============================================================================
# 3.고객 만족도를 라벨로 하여 다음과 같이 로지스틱 회귀분석을 수행하시오. 

# - 결측치가 포함된 행은 제거
# - 데이터를 7대 3으로 분리 (Seed = 123)
# - 아래의 11개 변수를 Feature로 사용
# Flight_Distance, Seat_comfort, Food_and_drink, Inflight_wifi_service, 
# Inflight_entertainment,Onboard_service, Leg_room_service, Baggage_handling,
# Cleanliness, Departure_Delay_in_Minutes, Arrival_Delay_in_Minutes
# 

# - Seed = 123, 이외의 항목은 모두 Default 사용
# - 예측 정확도를 측정하고 dissatisfied의 f1 score를 소수점 넷째 자리에서 반올림하여
# 소수점 셋째 자리까지 기술하시오. (답안 예시) 0.123
# =============================================================================

In [67]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [68]:
q3 = dataset.copy()

In [69]:
q3 = q3.dropna()

In [70]:
train, test = train_test_split(q3, test_size=0.3, random_state=123)

In [71]:
x_value = ['Flight_Distance', 'Seat_comfort', 'Food_and_drink', 'Inflight_wifi_service', \
'Inflight_entertainment','Onboard_service', 'Leg_room_service', 'Baggage_handling',\
'Cleanliness', 'Departure_Delay_in_Minutes', 'Arrival_Delay_in_Minutes']

In [72]:
lr = LogisticRegression(random_state=123).fit(train[x_value], train.satisfaction)

C:\ananconda\envs\multi\lib\site-packages\sklearn\linear_model\_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [73]:
pre = lr.predict(test[x_value])

In [83]:
metrics.accuracy_score(test.satisfaction, pre).round(3)

0.721

In [82]:
# pos_label => dissatisfied에 대하여 f1 score를 적용
metrics.f1_score(test.satisfaction, pre, pos_label='dissatisfied').round(3)

0.749